In [1]:
# !git clone https://github.com/JaidedAI/EasyOCR.git

In [2]:
# import os
# from glob import glob
# from tqdm import tqdm
# import shutil
# import pandas as pd

In [3]:
# for split in ['train', 'val', 'test']:
#     all_images = glob(f'./data/{split}/*')

#     # with open(f'./data/gt_{split}.txt', 'w') as file:
#     label_dict = {'filename': [], 'words': []}
#     for i, image_path in tqdm(enumerate(all_images)):
#         image_name = image_path.split('/')[-1]
#         image_label = image_name.split('_')[0]

#         # if image_label == '':
#         #     os.remove(image_path)
#         #     continue
            
#         new_image_name = f'{i}.' + image_name.split('.')[-1]
#         new_image_path = os.path.join('data', split, new_image_name)
#         shutil.move(image_path, new_image_path)
#         label_dict['filename'].append(new_image_name)
#         label_dict['words'].append(image_label)
        
#     df = pd.DataFrame(label_dict)
#     df.to_csv(f'data/{split}/labels.csv', index=None)

In [7]:
# !pip install -U pip
# !pip install -r EasyOCR/requirements.txt
# !pip install natsort

In [8]:
%cd EasyOCR/trainer

[Errno 2] No such file or directory: 'EasyOCR/trainer'
/home/ec2-user/SageMaker/ocr/EasyOCR/trainer


In [9]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [10]:
cudnn.benchmark = True
cudnn.deterministic = False

In [11]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [23]:
opt = get_config("config_files/ka_config.yaml")

In [24]:
train(opt, amp=True)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: data
opt.select_data: ['train', 'test', 'val']
opt.batch_ratio: ['1', '1', '1']
--------------------------------------------------------------------------------
dataset_root:    data	 dataset: train
data/train
sub-directory:	/train	 num samples: 147742
num total samples of train: 147742 x 1.0 (total_data_usage_ratio) = 147742
num samples of train per batch: 210 x 1.0 (batch_ratio) = 210
--------------------------------------------------------------------------------
dataset_root:    data	 dataset: test
data/test
sub-directory:	/test	 num samples: 2961
num total samples of test: 2961 x 1.0 (total_data_usage_ratio) = 2961
num samples of test per batch: 210 x 1.0 (batch_ratio) = 210
-------------------------------------------------------------------------

KeyboardInterrupt: 

In [42]:
opt = get_config("config_files/ka_config.yaml")

In [43]:
# opt

# Calculate accuracy 

In [45]:
!cat ./config_files/ka_config.yaml

number: '0123456789'
symbol: "."
lang_char: 'აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ'
experiment_name: 'ka_ocr_test'
train_data: 'data'
valid_data: 'data/test'
manualSeed: 36
workers: 4
batch_size: 256 #32
num_iter: 1
valInterval: 1
saved_model: 'saved_models/ka_ocr/best_norm_ED.pth'
FT: False
optim: False # default is Adadelta
lr: 1.
beta1: 0.9
rho: 0.95
eps: 0.00000001
grad_clip: 5
#Data processing
select_data: 'train-test' # 'train-test-val' # this is dataset folder in train_data
batch_ratio: '1-1' 
total_data_usage_ratio: 1.0
batch_max_length: 34 
imgH: 64
imgW: 600
rgb: False
contrast_adjust: False
sensitive: True
PAD: True
contrast_adjust: 0.0
data_filtering_off: False
# Model Architecture
Transformation: 'None'
FeatureExtraction: 'VGG'
SequenceModeling: 'BiLSTM'
Prediction: 'CTC'
num_fiducial: 20
input_channel: 1
output_channel: 256
hidden_size: 256
decode: 'greedy'
new_prediction: False
freeze_FeatureFxtraction: False
freeze_SequenceModeling: False

In [44]:
train(opt, amp=True)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: data
opt.select_data: ['train', 'test']
opt.batch_ratio: ['1', '1']
--------------------------------------------------------------------------------
dataset_root:    data	 dataset: train
data/train
sub-directory:	/train	 num samples: 147742
num total samples of train: 147742 x 1.0 (total_data_usage_ratio) = 147742
num samples of train per batch: 256 x 1.0 (batch_ratio) = 256
--------------------------------------------------------------------------------
dataset_root:    data	 dataset: test
data/test
sub-directory:	/test	 num samples: 2961
num total samples of test: 2961 x 1.0 (total_data_usage_ratio) = 2961
num samples of test per batch: 256 x 1.0 (batch_ratio) = 256
--------------------------------------------------------------------------------
Tota

SystemExit: 

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
